In [ ]:
import glob
import math
import numpy as np
import os
import random
import shutil
from PIL import Image

import tensorflow as tf

In [ ]:
names = {
    "iguchi": 0,
    "ushio": 1,
    "kakizaki": 2,
    "kageyama": 3,
    "kato": 4,
    "saito": 5,
    "sasaki_k": 6,
    "sasaki_m": 7,
    "takase": 8 ,
    "takamoto":9 ,
    "higashimura": 10,
    "kanemura": 11,
    "kawata": 12,
    "kosaka": 13,
    "tomita": 14,
    "nibu": 15,
    "hamagishi": 16,
    "matsuda": 17,
    "miyata": 18,
    "watanabe": 19,
    "kamimura": 20
}

In [ ]:
learning_path = "images/face_data/"
learning_member_path = glob.glob(learning_path + "*")

In [ ]:
learning_member_path = [p for p in learning_member_path if 'other' not in p]

In [ ]:
learning_path = "images/face_data/"
learning_member_path = glob.glob(learning_path + "*")
for index, member_path in enumerate(learning_member_path):
    learning_member_path[index] = member_path.replace("\\", '/')
    
    # それぞれのメンバーの学習用画像を全部取得
    file_list = glob.glob(member_path + "/" + "*")
    for image_file in file_list:
        os.makedirs(member_path + "/train/", exist_ok=True)
        shutil.move(image_file, member_path + "/train/" + image_file.split("\\")[-1])

In [ ]:
# move test folder
for member_path in learning_member_path:
    file_list = glob.glob(member_path + "/train/*")
    test_rate = 0.2
    test_num = math.floor(len(file_list) * test_rate)
    test_file_list = random.sample(file_list, test_num)
    
    for image_file in test_file_list:
        os.makedirs(member_path + "/test/", exist_ok=True)
        shutil.move(image_file, member_path + "/test/" + image_file.split("\\")[-1])

In [ ]:
size = (224,224)

In [ ]:
train_data_list = []
test_data_list = []
for memberFolder in learning_member_path:
    train_file_list = glob.glob(memberFolder + "/train/*")
    test_file_list = glob.glob(memberFolder + "/test/*")
    train_data_list.extend(train_file_list)
    test_data_list.extend(test_file_list)

In [ ]:
def CreateTensorflowReadFile(img_files, out_file):
    with tf.python_io.TFRecordWriter(out_file) as writer:
        for f in img_files:
            # ファイルを開く
            with Image.open(f) as image_object:
                image_object = image_object.resize(size, Image.LANCZOS)
                image = np.array(image_object)
                image = image[:,:,::-1]# RGB -> BGR
                 
                height = image.shape[0]
                width = image.shape[1]
                image_raw = image.tostring()
                label = names[f.split('/')[-2]] # ラベルを取得
 
                example = tf.train.Example(features=tf.train.Features(feature={
                        "height": tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
                        "width": tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_object.tobytes()])),
                        "path": tf.train.Feature(bytes_list=tf.train.BytesList(value=[f.encode('utf-8')]))
                        }))
 
            # レコード書込
            writer.write(example.SerializeToString())

In [ ]:
CreateTensorflowReadFile(train_data_list , "train_tf_file_%sx%s.tfrecords" % size) # 書き込み

In [ ]:
CreateTensorflowReadFile(test_data_list , "test_tf_file_%sx%s.tfrecords" % size) # 書き込み

In [ ]:
# 確認用
# output imagefile in tfrecord
count = 0
os.makedirs("check_tfRecords", exist_ok=True)
for record  in tf.python_io.tf_record_iterator("train_tf_file_%sx%s.tfrecords" % size):
    example = tf.train.Example()
    example.ParseFromString(record)  # バイナリデータからの読み込み
 
    height = example.features.feature["height"].int64_list.value[0]
    width = example.features.feature["width"].int64_list.value[0]
    label = example.features.feature["label"].int64_list.value[0]
    image = example.features.feature["image"].bytes_list.value[0]
    path = example.features.feature["path"].bytes_list.value[0]
 
    image = np.fromstring(image, dtype=np.uint8)
    image = image.reshape([height, width, 3])
    img = Image.fromarray(image, "RGB")
    img.save(os.path.join("check_tfRecords", "tfrecords_{0}-{1}.jpg".format(str(count), label)))
    count += 1